In [8]:
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import XGate, GlobalPhaseGate

The following cell creates Ship class that will be used in the game.

In [9]:
from random import randint
import numpy as np

#Ship Class
class Ship:
    def __init__(self, size, orientation, location):
        self.size = size

        if orientation == 'horizontal' or orientation == 'vertical':
            self.orientation = orientation
        else:
            raise ValueError("Value must be 'horizontal' or 'vertical'.")

        if orientation == 'horizontal':
            if location['row'] in range(row_size):
                self.coordinates = []
                for index in range(size):
                    if location['col'] + index in range(col_size):
                        self.coordinates.append({'row': location['row'], 'col': location['col'] + index})
                    else:
                        raise IndexError("Column is out of range.")
            else:
                raise IndexError("Row is out of range.")
        elif orientation == 'vertical':
            if location['col'] in range(col_size):
                self.coordinates = []
                for index in range(size):
                    if location['row'] + index in range(row_size):
                        self.coordinates.append({'row': location['row'] + index, 'col': location['col']})
                    else:
                        raise IndexError("Row is out of range.")
            else:
                raise IndexError("Column is out of range.")

        if self.filled():
            print_board(board)
            print(" ".join(str(coords) for coords in self.coordinates))
            raise IndexError("A ship already occupies that space.")
        else:
            self.fillBoard()

    def filled(self):
        for coords in self.coordinates:
            if board[coords['row']][coords['col']] == 1:
                return True
        return False

    def fillBoard(self):
        for coords in self.coordinates:
            board[coords['row']][coords['col']] = 1

    def contains(self, location):
        for coords in self.coordinates:
            if coords == location:
                return True
        return False

    # ─── inside the Ship class ─────────────────────────────────────────
    def destroyed(self, overlay=None):
        """
        Return True iff every square of this ship is marked 'X'.
        overlay:  players[i]['disp']  (live hit/miss grid).
                  If omitted, fall back to the old global board_display.
        """
        if overlay is None:                 # keep backward-compatibility
            overlay = board_display
    
        for sq in self.coordinates:
            mark = overlay[sq['row']][sq['col']]
            if mark == 'O':                 # still undiscovered
                return False
            if mark == '*':                 # miss on top of a ship → bug
                raise RuntimeError("board display inaccurate")
        return True
    # ───────────────────────────────────────────────────────────────────


    

The following cell if for setting the parameters of the game, such as the size of the board.

In [10]:
#Settings Variables
row_size = 4 #number of rows
col_size = 4 #number of columns
n_total= int(np.floor(np.log2(col_size*row_size)+1))
num_ships = 2
max_ship_size = 3
min_ship_size = 2
num_turns = 10

#Create lists
ship_list = []

board = [[0] * col_size for x in range(row_size)]

board_display = [["O"] * col_size for x in range(row_size)]

In [11]:
#Functions
def print_board(board_array):
    print("\n  " + " ".join(str(x) for x in range(1, col_size + 1)))
    for r in range(row_size):
        print(str(r + 1) + " " + " ".join(str(c) for c in board_array[r]))
    print()

def search_locations(size, orientation):
    locations = []

    if orientation != 'horizontal' and orientation != 'vertical':
        raise ValueError("Orientation must have a value of either 'horizontal' or 'vertical'.")

    if orientation == 'horizontal':
        if size <= col_size:
            for r in range(row_size):
                for c in range(col_size - size + 1):
                    if 1 not in board[r][c:c+size]:
                        locations.append({'row': r, 'col': c})
    elif orientation == 'vertical':
        if size <= row_size:
            for c in range(col_size):
                for r in range(row_size - size + 1):
                    if 1 not in [board[i][c] for i in range(r, r+size)]:
                        locations.append({'row': r, 'col': c})

    if not locations:
        return 'None'
    else:
        return locations

def random_location():
    size = randint(min_ship_size, max_ship_size)
    orientation = 'horizontal' if randint(0, 1) == 0 else 'vertical'

    locations = search_locations(size, orientation)
    if locations == 'None':
        return 'None'
    else:
        return {'location': locations[randint(0, len(locations) - 1)], 'size': size, \
                'orientation': orientation}

def get_row():
    while True:
        try:
            guess = int(input("Row Guess: "))
            if guess in range(1, row_size + 1):
                return guess - 1
            else:
                print("\nOops, that's not even in the ocean.")
        except ValueError:
            print("\nPlease enter a number")

def get_col():
    while True:
        try:
            guess = int(input("Column Guess: "))
            if guess in range(1, col_size + 1):
                return guess - 1
            else:
                print("\nOops, that's not even in the ocean.")
        except ValueError:
            print("\nPlease enter a number")

# Create the ships
temp = 0
while temp < num_ships:
    ship_info = random_location()
    if ship_info == 'None':
        continue
    else:
        ship_list.append(Ship(ship_info['size'], ship_info['orientation'], ship_info['location']))
        temp += 1
del temp
def merged_grid(pl):
    """Return board with disp-overlay (X, *, D)."""
    merged = [row[:] for row in pl['board']]          # copy ships/sea
    for r in range(row_size):
        for c in range(col_size):
            if pl['disp'][r][c] in ('X', '*', 'D'):   # only 3 dynamic marks
                merged[r][c] = pl['disp'][r][c]
    return merged

def repaint_own(pl):
    repaint(pl['own'], merged_grid(pl), own_colours)

def repaint_all():
    for pl in players:
        repaint_own(pl)                          # own board
        repaint(pl['atk'], pl['view'], view_colours)  # shot history


The core code for the classical battleship game was taken from the website [https://discuss.codecademy.com/t/excellent-battleship-game-written-in-python/430605](https://discuss.codecademy.com/t/excellent-battleship-game-written-in-python/430605).
The GUI was made with the help of ChatGPT.

### Quantum circuit

The following cell prepares a function that encodes the moves of the players on a quantum circuit and then simulates it to measure the outcome. Here we decided to have 4 coordinates for each round, which leads to 4 states. However, any power of 2 can be chosen as the number of coordinates/states.
For a given set of coordinates,
$$
(0, 0)_A(x_0, y_0)_B, \qquad (0, 0)_A(x_1, y_1)_B, \qquad (0, 0)_A(x_2, y_2)_B, \qquad (x_3, y_3)_A(0, 0)_B,
$$
with $(0, 0)$ meaning no attack, and the phases $\{\phi_0,\phi_1,\phi_2,\phi_3\}$, the following state will be prepared in the circuit.
$$
\vert\psi\rangle = e^{i\phi_0}\vert0, 0\rangle_A\vert x_0, y_0\rangle_B +
e^{i\phi_1}\vert0, 0\rangle_A\vert x_1, y_1\rangle_B +
e^{i\phi_2}\vert0, 0\rangle_A\vert x_2, y_2\rangle_B +
 e^{i\phi_3}\vert x_3, y_3\rangle_A\vert0, 0\rangle_B.
$$
The process of preparing the state requires post-selecting, therefore, the function runs single-shot simulations of the circuit, until the ancila read-out matches with the desired state. Then the result of the read-out of the registers A and B (state $\vert\psi\rangle$) will be returned.


In [12]:
from math import ceil, log2

def shot(a1_row, a1_col, theta1,
         a2_row, a2_col, theta2,
         def_row, def_col, theta3,
         atk1_row, atk1_col, theta4,
         atk2_row=-1, atk2_col=-1, theta5=0.0,
         defended=False):
    """
        This function gets a set of coordinates and the phases,
        and makes the quantum circuit and the quantum states of the shots. Then it starts to repeat single-shot
        simulation until the ancila qubits read-out is |00>. Then returns the result of the measurement of A and
        B registers.
    """
    global n_total, row_size, col_size  # From the upper scope
    
    # Turning coordinates from the format row/column to the number of the tile.
    # The first player
    coords, phases, fields = [], [], []
    coords += [a1_row*col_size + a1_col+1,  a2_row*col_size + a2_col+1]
    phases += [theta1*np.pi, theta2*np.pi]
    fields += [0, 0]
    
    # Changing the field of the target if it is a defence.
    if defended and def_row >= 0 and def_col >= 0:
        coords.append(def_row*col_size + def_col+1)
        phases.append(theta3*np.pi)
        fields.append(1)

    # The second player
    coords.append(atk1_row*col_size + atk1_col+1)
    phases.append(theta4*np.pi)
    fields.append(1)
    
    # Changing the field of the target if it is a defence.
    if atk2_row >= 0 and atk2_col >= 0:
        coords.append(atk2_row*col_size + atk2_col+1)
        phases.append(theta5*np.pi)
        fields.append(1)

    # Defining lists and parameters needed for computation
    states = len(coords)
    n_anc   = ceil(log2(states))
    pad_len = 2**n_anc - states
    coords += [0]*pad_len
    phases += [0.0]*pad_len
    fields += [0]*pad_len
    n = n_total
    
    def UGate(circuit, num, phase, qReg, controls):
        """
        This function receives the circuit, the coordinate and the phase, and the qubits,
        and applies the gates on the circuit to make the desired state.
        """
        num_bin = bin(num)[2:].zfill(n)  # Turning the coordinate to binary form
        # Applying X gates to make the state.
        for i in range(n):
            if num_bin[-i-1] == '1':
                qubits = [controls[j] for j in range(controls.size)] + [qReg[i]]
                x = XGate().control(controls.size)
                circuit.append(x, qubits)
        # Applying the desired phase
        GP = GlobalPhaseGate(phase)
        GP.num_qubits = 1
        qubits = [controls[j] for j in range(controls.size)] + [qReg[0]]
        circuit.append(GP.control(controls.size), qubits)
        return circuit
    
    N_ancilas = int(np.ceil(np.log2(states)))  # number of the ancila qubits
    
    # Prepating the registers
    ancila_qReg = QuantumRegister(N_ancilas, name='Qancila')
    qReg_A = QuantumRegister(n, name='qA')
    qReg_B = QuantumRegister(n, name='qB')
    ancila_cReg = ClassicalRegister(N_ancilas, name='Cancila')
    cReg_A = ClassicalRegister(n, name='cA')
    cReg_B = ClassicalRegister(n, name='cB')
    
    # Initializing the circuit
    circuit = QuantumCircuit(ancila_qReg, qReg_A, qReg_B, ancila_cReg, cReg_A, cReg_B)
    
    # H gates on the ancila qubits
    for i in range(N_ancilas):
        circuit.h(ancila_qReg[i])
    circuit.barrier()
    # Going through all the possible values of ancilas (e.g. |0,0>, |0,1>, |1,0>, |1,1>)
    # and applying the gates that make the desired states.
    for i in range(2**N_ancilas):
        bin_num = bin(i)[2:].zfill(N_ancilas)
        # Applying X gates to go over the ancila values
        for j in range(N_ancilas):
            if bin_num[j] == '1':
                circuit.x(ancila_qReg[-j-1])
        
        # Applying U gates to make the desired states
        if fields[i] == 0:
            circuit = UGate(circuit, coords[i], phases[i], qReg_A, ancila_qReg)
        else:
            circuit = UGate(circuit, coords[i], phases[i], qReg_B, ancila_qReg)

        #  Removing the X gates to turn it to |00>
        for j in range(N_ancilas):
            if bin_num[j] == '1':
                circuit.x(ancila_qReg[-j-1])
        circuit.barrier()
    
    #  Uncomputing H gates in the ancilas
    for i in range(N_ancilas):
        circuit.h(ancila_qReg[i])
    circuit.barrier()
    circuit.measure(ancila_qReg, ancila_cReg)
    circuit.measure(qReg_A, cReg_A)
    circuit.measure(qReg_B, cReg_B)

    
    # Prepatring the simulation
    sim     = AerSimulator(shots=1)
    circ    = transpile(circuit, sim)
    zeroTag = '0' * n_anc
    
    # repeating the measurement and post-selecting on ancilla = |0>
    while True:
        result = sim.run(circ).result()
        counts = result.get_counts()
        key = next(iter(counts))
        # Splitting the read-outs
        bits_B, bits_A, bits_anc = key.split(' ')
        if bits_anc == zeroTag:              # we keep only runs where ancilla is |0…0>
            break
    
    # Turning the binary outputs to decimal
    A = int(bits_A, 2)
    B = int(bits_B, 2)
    return A, B


### The game

The following cell runs the game. Enjoy! :D

In [ ]:
import tkinter as tk
from tkinter import simpledialog, messagebox

# ───────── helpers reused from last version ──────────────────────────
CELL = 36
own_colours = {
    0:"#d0e7ff",          # empty sea
    1:"#4f9cff",          # your ship
    'O':"#d0e7ff",        # undiscovered sea (same blue as 0)
    'X':"#ff4d4d",        # a hit
    '*':"#ffffff",        # a miss
    'D':"#66ff66",        # defender’s shield
}
view_colours = {'O':"#808080", 'X':"#ff4d4d", '*':"#ffffff",
                'P':"#bbbbbb", 'D':"#66ff66"}

# ───────── NEW helper functions (Fix ①) ─────────────────────────────
def merged_own_grid(pl):
    """board (0/1) overlaid with X * D marks from disp layer."""
    g = [row[:] for row in pl['board']]
    for r in range(row_size):
        for c in range(col_size):
            if pl['disp'][r][c] in ('X', '*', 'D'):
                g[r][c] = pl['disp'][r][c]
    return g

def draw_own(pl):
    repaint(pl['own'], merged_own_grid(pl), own_colours)

def idx_to_rc(idx):
    """Convert circuit index → (row, col).
       0  ➜  (-1,-1)   means “no hit”
       1  ➜  (0,0)     first tile
       .
    """
    if idx == 0:
        return -1, -1                 # sentinel = “nothing happened”
    idx -= 1                           # shift from 1-based to 0-based
    return idx // col_size, idx % col_size
# ────────────────────────────────────────────────────────────────────

def repaint(cv, grid, pal):
    cv.delete("all")
    for r in range(row_size):
        for c in range(col_size):
            x0, y0 = c*CELL, r*CELL
            cv.create_rectangle(x0, y0, x0+CELL, y0+CELL,
                                fill=pal[grid[r][c]], outline="black")

def hit(defender, r, c):
    for s in defender['ships']:
        if s.contains({'row': r, 'col': c}):
            defender['disp'][r][c] = 'X'
            if s.destroyed():
                defender['ships'].remove(s)
            return
    defender['disp'][r][c] = '*'

def build_fleet():
    global board, board_display
    board, board_display = (
        [[0]*col_size for _ in range(row_size)],
        [["O"]*col_size for _ in range(row_size)]
    )
    ships, placed = [], 0
    while placed < num_ships:
        info = random_location()
        if info == 'None':
            continue
        ship = Ship(info['size'], info['orientation'], info['location'])
        ships.append(ship)

        # ─── NEW: burn the ship’s squares into the board ───────────
        for sq in ship.coordinates:              # ← use .coordinates
            board[sq['row']][sq['col']] = 1  # 1 = my ship (dark blue)
        # ───────────────────────────────────────────────────────────

        placed += 1

    return {
        'board': board,
        'disp':  board_display,
        'ships': ships,
        'view':  [["O"]*col_size for _ in range(row_size)]
    }


# ───────── build two windows ─────────────────────────────────────────
p1, p2 = build_fleet(), build_fleet()
players = [p1, p2]

root = tk.Tk()
root.withdraw()
for idx, pl in enumerate(players, start=1):
    w = tk.Toplevel()
    w.title(str(idx))                        # “1” / “2”
    tk.Label(w, text="your fleet").grid(row=0, column=0)
    tk.Label(w, text="your shots").grid(row=0, column=1)

    own = tk.Canvas(w, width=col_size*CELL, height=row_size*CELL)
    atk = tk.Canvas(w, width=col_size*CELL, height=row_size*CELL)
    own.grid(row=1, column=0)
    atk.grid(row=1, column=1, padx=6)

    pl.update(win=w, own=own, atk=atk)
    draw_own(pl)                                # ← Fix ② (was repaint(...))
    repaint(atk, pl['view'], view_colours)

# ───────── round-state variables ─────────────────────────────────────
starter = 0                 # 0 or 1  (flips after each quantum step)
phase   = "A1"              # A1→A2→R_DEF/skip→R_ATK
data    = {}                # all coords + angles for current round

def prompt_angle(label, win):
    val = simpledialog.askfloat("secret", f"{label}/π (0–2)", parent=win)
    return val if val is not None else prompt_angle(label, win)
def in_board(r, c):
    return 0 <= r < row_size and 0 <= c < col_size
def finish_round():
    global phase, starter, data
    # fill missing keys so quantum_gate always gets all 16 params
    data.setdefault('def_row', -1)
    data.setdefault('def_col', -1)
    data.setdefault('theta3', 0.0)
    data.setdefault('atk2_row', -1)
    data.setdefault('atk2_col', -1)
    data.setdefault('theta5', 0.0)

    # call your circuit
    s1, s2 = shot(**data)

    # ← Fix ③: simple, correct conversion
    h1r, h1c = idx_to_rc(s1)
    h2r, h2c = idx_to_rc(s2)
    
    # allow only indices that are both ON the board and were ACTUALLY picked
    valid_starter   = {(data['a1_row'],  data['a1_col']),
                       (data['a2_row'],  data['a2_col'])}
    valid_responder = {(data['atk1_row'], data['atk1_col'])}
    if data.get('atk2_row', -1) >= 0:
        valid_responder.add((data['atk2_row'], data['atk2_col']))
    
    if not in_board(h1r, h1c) or (h1r, h1c) not in valid_starter:
        h1r = h1c = -1
    if not in_board(h2r, h2c) or (h2r, h2c) not in valid_responder:
        h2r = h2c = -1

    # apply hits
    hit(players[1-starter], h1r, h1c)
    hit(players[starter],   h2r, h2c)

    # update views
    players[starter]['view'][h1r][h1c]     = players[1-starter]['disp'][h1r][h1c]
    players[1-starter]['view'][h2r][h2c]   = players[starter]['disp'][h2r][h2c]

    # update views of the two returned squares  (already in your code)
    players[starter]['view'][h1r][h1c]   = players[1-starter]['disp'][h1r][h1c]
    players[1-starter]['view'][h2r][h2c] = players[starter]['disp'][h2r][h2c]

    # ─── NEW: clear all other 'P' so they can be picked next round ───
    for pl in players:
        for r in range(row_size):
            for c in range(col_size):
                if pl['view'][r][c] == 'P':
                    pl['view'][r][c] = 'O'
    # ────────────────────────────────────────────────────────────────

    # redraw
    for pl in players:
        draw_own(pl)
        repaint(pl['atk'], pl['view'], view_colours)
    # win?
    if not players[0]['ships'] or not players[1]['ships']:
        winner = "1" if players[1]['ships'] == [] else "2"
        messagebox.showinfo("game over", f"Player {winner} wins!")
        for pl in players:
            pl['win'].destroy()
        return

    # next round
    data.clear()
    starter = 1 - starter
    phase   = "A1"
    messagebox.showinfo("next round", f"New round — Player {starter+1} starts")
    defaults = dict(
        def_row=-1, def_col=-1, theta3=0.0,
        atk1_row=-1, atk1_col=-1, theta4=0.0,
        atk2_row=-1, atk2_col=-1, theta5=0.0,
        defended=False
    )
    for k, v in defaults.items():
        data.setdefault(k, v)

# ───────── NEW mouse handler ────────────────────────────────────────
def click(event):
    global phase, starter, data, row_size, col_size

    who  = 0 if event.widget in (p1['own'], p1['atk']) else 1
    me   = players[who]
    opp  = players[1-who]
    col, row = event.x // CELL, event.y // CELL
    if not (0 <= row < row_size and 0 <= col < col_size):
        return

    # (all your original state-machine logic is unchanged …)
    # ── starter attacks ────────────────────────────────────────────
    if phase == "A1" and who == starter and event.widget is me['atk']:
        if me['view'][row][col] in ('X', '*', 'P'):
            return
        θ1 = prompt_angle("θ₁", me['win'])
        data.update(a1_row=row, a1_col=col, theta1=θ1)
        me['view'][row][col] = 'P'
        repaint(me['atk'], me['view'], view_colours)
        phase = "A2"
        return

    if phase == "A2" and who == starter and event.widget is me['atk']:
        if me['view'][row][col] in ('X', '*', 'P'):
            return
        θ2 = prompt_angle("θ₂", me['win'])
        data.update(a2_row=row, a2_col=col, theta2=θ2)
        me['view'][row][col] = 'P'
        repaint(me['atk'], me['view'], view_colours)
        phase = "R_CHOICE"
        # assuming theta1 is already defined
        theta_min = θ2*np.pi - np.pi/3
        theta_max = θ2*np.pi + np.pi/3
        
        messagebox.showinfo(
            "Responder",
            f"Player {2 - starter}: choose (a) a defence tile on your board *or* "
            f"directly an attack tile on opponent's board.\n"
            f"The attack phase θ₁ must be between {theta_min:.2f} and {theta_max:.2f} radians."
        )
        return

    # ── responder’s first decision ─────────────────────────────────
    responder = 1 - starter
    if phase == "R_CHOICE" and who == responder:
        # ① chose to DEFEND
        if event.widget is me['own']:
            θ3 = prompt_angle("θ₃", me['win'])
            data.update(def_row=row, def_col=col, theta3=θ3, defended=True)
            me['disp'][row][col] = 'D'
            draw_own(me)                       # ← Fix ②
            phase = "R_ATK1"
            messagebox.showinfo(
                "attack",
                f"Player {who+1}: now pick ONE attack tile on opponent board"
            )
            return
        # ① chose first ATTACK  (double-attack route)
        if event.widget is me['atk']:
            if me['view'][row][col] in ('X', '*', 'P'):
                return
            θ4 = prompt_angle("θ₄", me['win'])
            data.update(atk1_row=row, atk1_col=col, theta4=θ4, defended=False)
            me['view'][row][col] = 'P'
            repaint(me['atk'], me['view'], view_colours)
            phase = "R_ATK2"
            messagebox.showinfo("second attack",
                                f"Player {who+1}: pick SECOND attack tile on opponent board")
            return

    # ── responder’s single attack when defence was chosen ──────────
    if phase == "R_ATK1" and who == responder and event.widget is me['atk']:
        if me['view'][row][col] in ('X', '*', 'P'):
            return
        θ4 = prompt_angle("θ₄", me['win'])
        data.update(atk1_row=row, atk1_col=col, theta4=θ4,
                    atk2_row=-1, atk2_col=-1, theta5=0.0)
        me['view'][row][col] = 'P'
        repaint(me['atk'], me['view'], view_colours)
        finish_round()
        return

    # ── responder’s SECOND attack when no defence ──────────────────
    if phase == "R_ATK2" and who == responder and event.widget is me['atk']:
        if me['view'][row][col] in ('X', '*', 'P'):
            return
        θ5 = prompt_angle("θ₅", me['win'])
        data.update(atk2_row=row, atk2_col=col, theta5=θ5)
        me['view'][row][col] = 'P'
        repaint(me['atk'], me['view'], view_colours)
        finish_round()
        return

# bind all canvases
for pl in players:
    pl['own'].bind("<Button-1>", click)
    pl['atk'].bind("<Button-1>", click)

root.mainloop()
